In [ ]:
import re
from settings import *

text = '\n   aa   e483zigko zez\n\nef\n j •j •  j.  dz...  $;.  \n<eom><eod>\n\n<eom>  \t  %kzeé i3zçr zg   35  ¨e:=zge \ng   ezù5 =   = zae 5gez $gez  \n'

text = text.replace('\n', '<nl>').replace('\t', '    ')

# Start

# Split control characters

words = [text]
temp = []

for control in CONTROL_CHARS:
	for i in range(len(words)):

		parts = words[i].split(control)

		for part in parts:
			if len(part) > 0:
				temp.append(part)

			temp.append(control)

		if len(parts) > 0:
			temp.pop()

	words = temp
	temp = []

# Split words

for i in range(len(words)):

	if words[i] in CONTROL_CHARS:
		temp.append(words[i])
		continue

	word = ''

	for j in range(len(words[i])):

		if len(word) == 0:
			word += words[i][j]
			continue

		if word[-1] != ' ' and words[i][j] == ' ':
			temp.append(word)
			word = words[i][j]
			continue

		if word[-1].isalpha() and not words[i][j].isalpha():
			temp.append(word)
			word = words[i][j]
			continue

		if word[-1] != ' ' and not word[-1].isalpha() and words[i][j].isalpha():
			temp.append(word)
			word = words[i][j]
			continue

		word += words[i][j]

	if len(word) > 0:
		temp.append(word)

words = temp
temp = []

# Split multiple spaces

for i in range(len(words)):

	j = 0

	while j < len(words[i]) and words[i][j] == ' ':
		j += 1

	if j > 1 and j < len(words[i]):
		temp.append(words[i][:j - 1])
		temp.append(words[i][j - 1:])

	else:
		temp.append(words[i])

words = temp
temp = []

# Stop

print(text.replace(' ', '_'))

for i in range(len(words)):
	if len(words[i]) == 0:
		print('\033[92m_\033[0m', end = '')
	if i % 2 == 1:
		print('\033[91m' + words[i].replace(' ', '_') + '\033[0m', end = '')
	else:
		print('\033[94m' + words[i].replace(' ', '_') + '\033[0m', end = '')

In [ ]:
print('a.n'.split('.'))

In [ ]:
import time
import random
import string

def get_random_string():

	return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(10))

a = {}

for i in range(1000000):
	a[get_random_string()] = 0

a['aeqhrtherqth'] = 0
b = {'aeqhrtherqth': 0}

In [ ]:
start = time.time()
print('aeqhrtherqth' in a)
end = time.time()
print(end - start)

start = time.time()
print('aeqhrtherqth' in b)
end = time.time()
print(end - start)

In [ ]:
import re
import regex
import utils
from settings import *
a = '   aa   e483zigko ze<nl>zef j <nl>•j •<nl>  j.  dz...ze____%ç __ '$;.  <eom><eod><eom> a___ef__  >=0 %kzeé i3zçr zg   35  ¨e:=zge g   ezù5 =   = zae 5gez $gez   '

# b = regex.split('(' + '|'.join(CONTROL_CHARS) + ')', a, flags = regex.UNICODE)
# b = list(filter(None, b))

# temp = []

# for i in b:

# 	if i in CONTROL_CHARS:
# 		temp.append(i)
# 		continue

# 	t = regex.split(r'(\d+|\s+|\p{L}+|[^\d\p{L}\s]+)', i, flags = regex.UNICODE)
# 	t = list(filter(None, t))
# 	temp.extend(t)

# b = temp
reg = r'(' + r'|'.join(CONTROL_CHARS) + r'|\d+|\s+|\p{L}+|[^\d\p{L}\s' + r''.join([f'[{i}]' for i in CONTROL_CHARS]) + r']+)'
b = regex.split(reg, a, flags = regex.UNICODE)
b = list(filter(None, b))

'''
temp = []
i = 0

while i < len(b) - 1:

	if b[i] == ' ':
		temp.append(' ' + b[i + 1])
		i += 2
		continue

	if b[i].endswith(' '):
		temp.extend([b[i][:-1], ' ' + b[i + 1]])
		i += 2
		continue

	temp.append(b[i])
	i += 1

temp.append(b[-1])

b = temp
'''
print(a)
utils.print_tokens(b)

In [ ]:
print('0'.isdecimal())

In [8]:
import numpy as np

a = np.random.normal(0.5, 2, 100)

# apply softmax
b = np.exp(a) / np.sum(np.exp(a))

probabilities = b
print(list(probabilities))
probabilities = np.log(probabilities) / 0.5
print(list(probabilities))
probabilities = np.exp(probabilities) / np.sum(np.exp(probabilities))
print(list(probabilities))

sorted_indices = np.argsort(probabilities)[::-1]
print(list(sorted_indices))
cumulative_probabilities = np.cumsum(probabilities[sorted_indices])
print(list(cumulative_probabilities))
sorted_indices = sorted_indices[cumulative_probabilities <= 1.0]
print(list(sorted_indices))
probabilities = probabilities[sorted_indices]
print(list(probabilities))
probabilities = probabilities / np.sum(probabilities)
print(list(probabilities))

[0.013606817106636838, 0.01623092182219026, 0.0006562823047288122, 0.00043289442970814837, 0.00012074408943789908, 0.1572751901834894, 0.00047341713478700936, 0.001039923501874295, 0.0004294272014198699, 0.0007745368027274086, 8.421868422058007e-06, 0.0027963547787567223, 0.0034540137820744067, 0.0004894518780543952, 0.0002894165469550305, 0.002892466471674889, 0.0009870866390823752, 0.004209481632345724, 0.0015114450977574295, 0.0018425479392781949, 0.000950520297247409, 0.007100253864676865, 0.0031501776189201427, 0.0038734351497773083, 0.0012739252462064595, 0.0027813148072126947, 0.0025580492503807124, 0.0019495886679201195, 0.0038657237230106016, 0.017136899822737705, 0.0008046023482882162, 4.392766544639197e-05, 0.0017383390713697306, 0.0013177162436547993, 9.392035454077647e-05, 0.0006713110474122766, 0.00013674714501585528, 0.00013929722689980825, 0.0008974325635182707, 0.06519842423908584, 0.004467711073980535, 0.006579441949715373, 0.0005526977604351253, 0.0015142428094831945